In [113]:
import requests
import wget
import json
from os.path import isfile, join
from os import listdir
import pandas as pd
import arrow
from matplotlib import pyplot as plt

In [114]:
with open('config/demroes_config.json', 'r') as fp:
    config = json.load(fp)
with open('metadata/raw_demroes_metadata.json', 'r') as fp:
    metadata_dict = json.load(fp)
with open('metadata/demroes_point_map.json', 'r') as fp:
    point_map = json.load(fp)
with open('metadata/demroes_unit_map.json', 'r') as fp:
    unit_map = json.load(fp)

In [115]:
for building_name, item in config.items():
    url = item['url']
    filename = 'raw_data/' + url.split('/')[-1]
    wget.download(url, out=filename)

In [116]:
raw_data_dir = './raw_data/'
files = [f for f in listdir(raw_data_dir) if isfile(join(raw_data_dir, f))]

In [117]:
datadir = './data/'
new_metadata_dict = dict()
for f in files:
    filename = raw_data_dir + f
    df = pd.read_csv(filename)
    time_list = [arrow.get(arrow.get(timestamp, 'MM/DD/YY HH:mm').datetime, 'US/Pacific') \
                 for timestamp in df['Timestamp'].tolist()]
    for data_name, column in df.iteritems():
        point_type = point_map.get(data_name)
        if not point_type:
            continue
        new_metadata = dict()
        srcid = f.split('.')[0] + '_' + point_type
        value_list = column.tolist()
        data = pd.Series(index=time_list, data=value_list)
        data.to_csv(datadir+srcid+'.csv', index_label='time', header=['value'])
        new_metadata['point_type'] = point_type
        new_metadata['unit'] = unit_map[data_name]
        new_metadata['name'] = srcid
        building_id = f.split('_')[0].lower()
        new_metadata['latitude'] = metadata_dict[building_id]['location']['latitude']
        new_metadata['longitude'] = metadata_dict[building_id]['location']['longitude']
        new_metadata_dict[srcid] = new_metadata        

In [118]:
for f in files:
    filename = raw_data_dir + f
    os.remove(filename)

In [119]:
with open('metadata/demroes_metadata.json', 'w') as fp:
    json.dump(new_metadata_dict, fp)